In [1]:
import pandas as pd
import janitor

In [2]:
oct_dm = pd.read_csv("/Users/yukachen/marketing-operation/direct-mail/2024-october/csv-marketing_campaign_members-2024-11-12-21-51-51-9390.csv")

In [3]:
oct_sfdc_data = pd.read_csv("/Users/yukachen/marketing-operation/direct-mail/2024-october/2024-october-sfdc-leads.csv")

In [23]:
oct_sam_list_a_western = pd.read_csv("/Users/yukachen/marketing-operation/direct-mail/2024-october/October 2024 DM SAM List - A - Western 1-26130.csv")
oct_sam_list_b_chinese = pd.read_csv("/Users/yukachen/marketing-operation/direct-mail/2024-october/October 2024 DM SAM List - B - Chinese 26131-51680.csv")

In [24]:
oct_sam_list_a_western['test_group'] = "A - Western Style"
oct_sam_list_b_chinese['test_group'] = "B - Chinese Style"

oct_sam_list = pd.concat([oct_sam_list_a_western, oct_sam_list_b_chinese], ignore_index=True)

oct_sam_list = oct_sam_list.clean_names()

del oct_sam_list_a_western, oct_sam_list_b_chinese

In [ ]:
oct_sfdc_data['mobile_primary_sfdc'] = oct_sfdc_data['mobile_primary_sfdc'].astype(str)
oct_sfdc_data['business_phone_sfdc'] = oct_sfdc_data['business_phone_sfdc'].astype(str)

oct_sam_list['phone'] = oct_sam_list['phone'].astype(str)
oct_sam_list_unique = oct_sam_list.drop_duplicates(subset=['phone'])

oct_lead_brizo_matched2 = pd.merge(oct_sfdc_data, oct_sam_list_unique, left_on='mobile_primary_sfdc', right_on='phone', how='inner')
oct_lead_brizo_matched3 = pd.merge(oct_sfdc_data, oct_sam_list_unique, left_on='business_phone_sfdc', right_on='phone', how='inner')

oct_lead_brizo_matched = pd.concat([oct_lead_brizo_matched2, oct_lead_brizo_matched3], ignore_index=True)

oct_lead_brizo_unmatched = pd.merge(oct_sfdc_data, oct_lead_brizo_matched, left_on='')


In [ ]:
del oct_lead_brizo_matched2, oct_lead_brizo_matched3, oct_sam_list


In [7]:
scans_id = pd.read_csv("/Users/yukachen/marketing-operation/direct-mail/2024-october/Google Analytics Report (Daily Updates) - Q4 QR Scans.csv")

selected_scans_id = scans_id[['date_c', 'campaign_c', 'unique_id','region','country']].dropna(subset='unique_id')

del scans_id



In [8]:
joined_scans_w_brizo = pd.merge(selected_scans_id, oct_sam_list, left_on='unique_id', right_on='snowball_map', how='left')

del selected_scans_id

In [9]:
print(joined_scans_w_brizo.columns)

Index(['date_c', 'campaign_c', 'unique_id', 'region', 'country_x',
       'snowball_map', 'brizo_id', 'name', 'street', 'city_town',
       'zip_postal_code', 'state_province_code', 'country_y',
       'cuisines_regional_', 'price_range', 'opening_hours', 'vitality_score',
       'established', 'status', 'phone', 'most_common_email', 'density_@5mi',
       'time_zone_name', 'median_price', 'full_name', 'role', 'test_group'],
      dtype='object')


In [10]:
selected_scans_brizo = joined_scans_w_brizo[['unique_id','cuisines_regional_','date_c']]

del joined_scans_w_brizo

In [11]:
scans_summary = selected_scans_brizo.groupby('cuisines_regional_').agg(
    value_count=('unique_id', 'count')
).reset_index().sort_values(by='value_count', ascending=False)

del selected_scans_brizo, scans_summary